In [694]:
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell

from data_loader import get_pdf_filepaths, load_page_and_line_indexes, load_split_data, load_raw_indexes_list
from pdf_reader import parse_pdf
from data_cleaner import clean_initial_indexes, add_split_data
from data_saver import save_page_and_line_indexes, save_split_data, save_raw_indexes_list, save_aggregated_data
from data_transformer import (
  get_candidates_and_frequencies, 
  add_frequencies_column, 
  add_is_in_toc, 
  add_importance,
  add_position_in_context,
  add_is_named_entity,
  add_length_of_word,
  add_is_named_author,
  add_tfidf,
  get_raw_indexes_list,
  add_is_in_index,
  aggregate_by_candidate
)


#InteractiveShell.ast_node_interactivity = "all"

# Configuration variables

In [33]:
DATA_DIR_PATH = "../data/"
PDF_SOURCE_DIR_PATH = DATA_DIR_PATH + "pdf/"
PROCESSED_DATA_DIR_PATH = DATA_DIR_PATH + "processed/"

# Load data

In [34]:
file_paths = get_pdf_filepaths(PDF_SOURCE_DIR_PATH)
file_paths

['/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2017_Book_TheDataScienceDesignManual.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/morris_07.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/lyons_96.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2015_Book_LinearAlgebra.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/christensen_04.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/keefe_00.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/linnebo_18.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/oliver_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/jenkins_08.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/zimmermann_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/shapiro_9

# Raw line and page data


In [695]:

for file_path in file_paths[10:11]:
    raw_line_and_page_indexes = parse_pdf(file_path)

    line_and_page_indexes = clean_initial_indexes(raw_line_and_page_indexes)

  # line_and_page_indexes['file_name']
  # line_and_page_indexes['by_line'].head()
  # line_and_page_indexes['by_page'].head()

    save_page_and_line_indexes(
     processed_data_dir_path=PROCESSED_DATA_DIR_PATH, 
     line_and_page_indexes=line_and_page_indexes
    )


# Data set split


In [696]:
file_path = file_paths[10]

line_and_page_indexes = load_page_and_line_indexes(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)
with_split_data = add_split_data(
  file_path=file_path,
  line_and_page_indexes=line_and_page_indexes
)

# with_split_data['by_page_body'].head()

save_split_data(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  split_data=with_split_data  
)
 


# Prepare input data frames

In [697]:
with_split_data['by_page_toc']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3,clean_content
0,"Contents\n\nIntroduction\n\n3\n\nPART I PERSPECTIVE\n\n1 Mathematics and Its Philosophy 21\n2 Object and Truth: A Realist Manifesto 36\n1 Slogans 36\n2 Methodology 38\n3 Philosophy 44\n4 Interlude on Antirealism 51\n5 Quine 52\n6 A Role for the External 57\nPART II STRUCTURALISM\n\n3\n\nStructure 71\n1 Opening 71\n2 Ontology: Object 77\n3 Ontology: Structure 84\n4 Theories of Structure 90\n5 Mathematics: Structures, All the Way Down 97\n6 Addendum: Function and Structure 106\n\n4 Epistemology and Reference 109\n1 Epistemic Preamble 109\n2 Small Finite Structure: Abstraction and Pattern Recognition 112",10,10,Contents,,,content introduction part perspective mathematics philosophy object truth realist manifesto slogan methodology philosophy interlude antirealism quine role external part ii structuralism structure opening ontology object ontology structure theory structure mathematics structure way addendum function structure epistemology reference epistemic preamble small finite structure abstraction pattern recognition
1,"x CONTENTS\n\n3\n4\n5\n6\n7\n8\n9\n\nLong Strings and Large Natural Numbers 116\nTo the Infinite: The Natural-number Structure 118\nIndiscernibility, Identity, and Object 120\nOntological Interlude 126\nImplicit Definition and Structure 128\nExistence and Uniqueness: Coherence and Categoricity 132\nConclusions: Language, Reference, and Deduction 136\n\n5 How We Got Here 143\n1 When Does Structuralism Begin? 143\n2 Geometry, Space, Structure 144\n3 A Tale of Two Debates 152\n4 Dedekind and ante rem Structures 170\n5 Nicholas Bourbaki 176\nPART III\n\nRAMIFICATIONS AND APPLICATIONS\n\n6 Practice: Construction, Modality, Logic 181\n1 Dynamic Language 181\n2 Idealization to the Max 183\n3 Construction, Semantics, and Ontology 185\n4 Construction, Logic, and Object 189\n5 Dynamic Language and Structure 193\n6 Synthesis 198\n7 Assertion, Modality, and Truth 203\n8 Practice, Logic, and Metaphysics 211\n7 Modality, Structure, Ontology 216\n1 Modality 216\n2 Modal Fictionalism 219\n3 Modal Structuralism 228\n4 Other Bargains 230\n5 What Is a Structuralist to Make of All This? 235\n8 Life Outside Mathematics: Structure and Reality 243\n1 Structure and Science—the Problem 243\n2 Application and Structure 247\n3 Borders 255\n4 Maybe It Is Structures All the Way Down 256\nReferences\nIndex\n\n273\n\n263",11,11,Contents,,,x content long string large natural number infinite natural-number structure indiscernibility identity object ontological interlude implicit definition structure existence uniqueness coherence categoricity conclusion language reference deduction got structuralism begin geometry space structure tale two debate dedekind ante rem structure nicholas bourbaki part iii ramification application practice construction modality logic dynamic language idealization max construction semantics ontology construction logic object dynamic language structure synthesis assertion modality truth practice logic metaphysics modality structure ontology modality modal fictionalism modal structuralism bargain structuralist make life outside mathematics structure reality structure science-the problem application structure border maybe structure way reference index
2,PHILOSOPHY of\nMATHEMATICS,12,12,Contents,,,philosophy mathematics


In [698]:
with_split_data['by_page_biblio']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3
0,"References\n\nAspray, W., and P. Kitcher [1988], History and philosophy of modern mathematics, Minnesota studies in the philosophy of science 11, Minneapolis, Minnesota University Press.\nAzzouni, J. [1994], Metaphysical myths, mathematical practice, Cambridge, Cambridge\nUniversity Press.\nBalaguer, M. [1994], “Against (Maddian) naturalized epistemology,” Philosophia Mathematica (3) 2: 97–108.\nBalaguer, M. [1995], “A Platonist epistemology,” Synthese 103: 303–325.\nBarbut, M. [1970], “On the meaning of the word ‘structure’ in mathematics,” in Introduction to structuralism, ed. by M. Lane, New York, Basic Books, 367–388.\nBarwise, J. [1985], “Model-theoretic logics: Background and aims,” in Model-theoretic logics, ed. by J. Barwise and S. Feferman, New York, Springer-Verlag, 3–23.\nBenacerraf, P. [1965], “What numbers could not be,” Philosophical Review 74: 47–73; reprinted in Benacerraf and Putnam [1983], 272–294.\nBenacerraf, P. [1973], “Mathematical truth,” Journal of Philosophy 70: 661–679; reprinted in Benacerraf and Putnam [1983], 403–420.\nBenacerraf, P., and H. Putnam [1983], Philosophy of mathematics, second edition, Cambridge, Cambridge University Press.\nBernays, P. [1935], “Sur le platonisme dans les mathématiques,” L’enseignement mathématique 34: 52–69; tr. as “Platonism in mathematics,” in Benacerraf and Putnam [1983], 258–\n271.\nBernays, P. [1961], “Zur Frage der Unendlichkeitsschemata in der axiomatischen Mengenlehre,” in Essays on the foundation of mathematics, ed. by Y. Bar-Hillel et al., Jerusalem,\nMagnes Press, 3–49.\nBernays, P. [1967], “Hilbert, David,” in The encyclopedia of philosophy, vol.\n3. ed. by\nP. Edwards, New York, Macmillan and The Free Press, 496–504.\nBeth, E. W., and J. Piaget [1966], Mathematical epistemology and psychology, Dordrecht,\nHolland, D. Reidel.\nBishop, E. [1967], Foundations of constructive analysis, New York, McGraw-Hill.\nBishop, E. [1975], “The crises in contemporary mathematics,” Historia Mathematica 2:\n505–517.\n263",274,274,References,,
1,"264 REFERENCES\nBlackburn, S. [1984], Spreading the word, Oxford, Clarendon Press.\nBlock, N. (ed.) [1980], Readings in philosophy of psychology 1, Cambridge, Harvard\nUniversity Press.\nBoolos, G. [1984], “To be is to be a value of a variable (or to be some values of some variables),” Journal of Philosophy 81: 430–449.\nBoolos, G. [1985], “Nominalist Platonism,” Philosophical Review 94: 327–344.\nBoolos, G. [1987], “The consistency of Frege’s Foundations of arithmetic,” in On being and saying: Essays for Richard Cartwright, ed. by Judith Jarvis Thompson, Cambridge, MIT\nPress, 3–20.\nBourbaki, N. [1949], “Foundations of mathematics for the working mathematician,” Journal of Symbolic Logic 14: 1–8.\nBourbaki, N. [1950], “The architecture of mathematics,” American Mathematical Monthly\n57: 221–232.\nBourbaki, N. [1968], Theory of sets, Paris, Hermann.\nBrouwer, L. E. J. [1948], “Consciousness, philosophy and mathematics,” in Benacerraf and Putnam [1983], 90–96.\nBuck, R. [1956], Advanced calculus, New York, McGraw-Hill.\nBurgess, J. [1983], “Why I am not a nominalist,” Notre Dame Journal of Formal Logic\n24: 93–105.\nBurgess, J. [1984], “Synthetic mechanics,” Journal of Philosophical Logic 13: 379–395.\nBurgess, J. [1992], “Proofs about proofs: A defense of classical logic,” in Proof, logic and formalization, ed. by Michael Detlefsen, London, Routledge, 8–23.\nCantor, G. [1899], “Letter to Dedekind,” van Heijenoort [1967], 113–117.\nCantor, G. [1932], Gesammelte Abhandlungen mathematischen und philosophischen\nInhalts, ed. by E. Zermelo, Berlin, Springer.\nCarnap, R. [1931], “Die logizistische Grundlegung der Mathematik,” Erkenntnis 2: 91–105; tr. as “The logicist foundations of mathematics,” in Benacerraf and Putnam [1983], 41–52.\nCarnap, R. [1934], Logische Syntax der Sprache, Vienna, Springer; tr. as The logical syntax of language, New York, Ha

In [699]:
with_split_data['by_page_index']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3
0,"INDEX\n\n1\n\n273\n\nIndex a posteriori knowledge, see a priori knowledge a priori knowledge, 111, 112, 116, 124,\n132, 141, 255 in geometry, 144–145, 156, 163n.14 abstract object, 10, 16, 74–76, 87, 101,\n109–116, 124, 141–142, 216, 233, 248 abstract/concrete dichotomy, 109, 111 abstract structure, see algebraic theory abstraction, 11–12, 74–75, 111–124, 172–\n175 acceptability thesis, 167–168 actual infinity, see potential infinity algebraic theory, 40–41, 50, 73n.2, 133\nAmerican postulate theorists, 160–161, 176 analysis, real, see real analysis analytic truth, 58, 85, 111, 209n.20 ancestral, 166–167, 171n.21 ante rem option, 89–90, 92–97, see also structuralism, ante rem ante rem structure, see structuralism, ante rem antirealism, see realism, antiapplications of mathematics, 4–5, 17, 32,\n34, 45–46, 60–61, 115–116, 169n.18,\n243–254, see also geometry; indispensability\nAristotle, 84 assertabilism, 203–211 axiom of choice, see choice, axiom of axiomatics, 133–136, 149–150, 157–165,\n176–177, see also implicit definition\nAzzouni, J., 82\nBaire, R., 24, 25, 38, 39\nBalaguer, M., 86–87n.12, 134 baseball, 9, 74, 76, 78, 79–80, 98–100,\n101, 119, 129, 258\nBelnap, N., 49n.12\nBeltrami, E., 150\nBenacerraf, P., 3–6, 18, 45, 78–81, 86,\n109–110, 146, 172, 258, 260n.10\nBerkeley, G., 33, 50n.14\nBernays, P., 26–27, 31, 39, 95n.18, 158\nBeth, E., 188\nBishop, E., 23, 26–27, 187\nBlackburn, S., 208, 217\nBlock, N., 106–108\nBolzano, B., 31, 144\nBolzano-Weierstrass theorem, 184, 187–188\nBoolos, G., 78n.6, 105, 233–235, 242\nBorel, E., 24, 25, 38, 39\nBourbaki, N., 176–177, 248 bridge principles, 220, 221, 228, 246–247\nBrouwer, L. E. J., 22–23, 37, 190n.7, 198,\n200–201, 208\nBurgess, J., 29, 238n.17\nCaesar problem, 78–81, 127, 131, 163–\n164, 168–170, 175, 258\n\n273",284,284,References,,
1,"274 INDEX\nCantor, G., 24, 28, 116, 171, 174–175, 202\nCantor’s theorem, 127\nCarnap, R., 28–29, 58–61, 144, 192n.8 categoricity, 13, 77, 131, 132–133, 140,\n159–160, 174, 221–222, 228–229, see also isomorphism category theory, 87–88, 93, 96, 192\nCauchy sequence, see real analysis causal theory of knowledge, 45, 109–110,\n111, 112\nCH, see continuum hypothesis\nChasles, M., 148\nChihara, C., 37, 86–87n.12, 196–197n.10,\n201n.12, 217, 230–233, 237, 240,\n247n.5, 250 choice, axiom of, 24–26, 38–39, 42, 188\nChomsky, N., 250\nChurch, A., 161\nChurch’s thesis, 135n.20, 209, 240–241, 251 class, proper, see proper classes classical logic, 15, 22–27, 30n.11, 38–44,\n48–49, 54, 119, 127, 173, 182–183,\n185–212, see also revisionism\nCoffa, A., 144, 152–153, 154n.8, 156, 161,\n165 coherence (of structure definitions), 13, 95,\n105, 118, 131, 133–136, 167–168,\n226, 258–261 coherence principle, 95, 105, 133–134\nCohn-Vossen, S., 158n.10 completeness, 26n.8, 31, 95, 134–135,\n138n.25, 140, 221–222, 224, 231n.13 complex number, 33, 81, 102, 145–146,\n249–250 concrete structure, see algebraic theory congruence relation, 93, 122–124, 127, 128 consequence, 17, 62, 122, 132, 149, 155,\n158n.10, 161, 162–164, 185–189,\n213–215, 216–217, 219, 221–224,\n227–228, 238, see also logical possibility; model theory conservativeness, 219–222, 227–228 consistency, 13, 47, 78n.16, 95, 134–136,\n138n.25, 158–159, 161–165, 167, 202,\n223, see also logical possibility constructibility quantifiers, 230–233, 236–\n238 constructive mathematics, see classical logic context principle, 14, 166–170 continuity (in geometry and analysis),\n102n.23, 138–139, 145, 170–171, 174,\n183n.2, 200–201 continuum hypothesis, 40–42, 217, 220–\n221, 231–232, 245 convention T, 48, see also Tarskian semantics\nCooper, R., 189–190 correspondence theory of truth, 63, 64n.19,\n66–67, see also truth\nCorry, L., 176–177\nCrossley, J., 184n.3, 188\nCrowell, R., 251, 255\nCurry, H., 51, 53, 56\nDavidson, D., 47–48, 62, 63, 191\nDedekind, R., 14, 26, 39, 102n.23,\n169n.18, 170–176, 239\nDedekind cut, see real analys

In [700]:


split_data = load_split_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    pdf_filepath=file_path
  )
# split_data['by_line_toc'].head()

(candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

# for file_path in file_paths[3:4]:
#   split_data = load_split_data(
#     processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
#     pdf_filepath=file_path
#   )
#   split_data['by_line_toc'].head()

#   (candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

#   candidates_df.head()

#   with_frequencies = add_frequencies_column(
#     by_pages_body_df=split_data['by_page_body'],
#     candidates_df=candidates_df,
#     freq_ngrams=freq_ngrams
#   )

#   with_frequencies.head()

#candidates_df.head(30)

In [661]:
with_frequencies = add_frequencies_column(
  by_pages_body_df=split_data['by_page_body'],
  candidates_df=candidates_df,
  freq_ngrams=freq_ngrams
)

In [662]:
with_is_in_toc = add_is_in_toc(
  candidates_df=with_frequencies, 
  by_line_toc=split_data['by_line_toc']
)

#with_is_in_toc.head(30)


In [666]:
with_position_in_context = add_position_in_context(with_is_in_toc)



In [667]:
with_importance = add_importance(with_position_in_context)

In [668]:
with_is_named_entity = add_is_named_entity(
  candidates_df=candidates_df,
  df_pages_body=split_data['by_page_body']
)

In [669]:
with_length_of_word = add_length_of_word(candidates_df)

In [670]:
with_is_named_author = add_is_named_author(
  candidates_df=with_length_of_word,
  df_pages_biblio=split_data['by_page_biblio']
)

In [671]:
with_tfidf = add_tfidf(
  candidates_df=with_is_named_author,
  df_pages_body=split_data['by_page_body']
)

In [672]:
with_tfidf.head()

,candidate_keyword,clean_context,raw_context,POS,freq,is_in_toc,position_in_context,importance,is_named_entity,length,is_named_author,tfidf
0,introduction,introduction,introduction,NOUN,0.000240,1,0.000000,0.099739,0,12,0,0.095556
1,project,project,the project,NOUN,0.000327,1,1.000000,0.041221,0,7,0,0.121081
2,goal,goal book locate attractive option philosophical space new kind arithmetical epistemology one respect certain important intuition hitherto considered tension impossible satisfy simultaneously,my goal in this book is to locate a an attractive option in philosophical space a new kind of arithmetical epistemology one which respect certain important intuition hitherto considered to be in tension and impossible to satisfy simultaneously,NOUN,0.000038,0,0.027027,0.893139,0,4,0,0.045532
3,book,goal book locate attractive option philosophical space new kind arithmetical epistemology one respect certain important intuition hitherto considered tension impossible satisfy simultaneously,my goal in this book is to locate a an attractive option in philosophical space a new kind of arithmetical epistemology one which respect certain important intuition hitherto considered to be in tension and impossible to satisfy simultaneously,NOUN,0.000356,0,0.108108,0.893139,0,4,0,0.118328
4,locate,goal book locate attractive option philosophical space new kind arithmetical epistemology one respect certain important intuition hitherto considered tension impossible satisfy simultaneously,my goal in this book is to locate a an attractive option in philosophical space a new kind of arithmetical epistemology one which respect certain important intuition hitherto considered to be in tension and impossible to satisfy simultaneously,VERB,0.000010,0,0.189189,0.893139,0,6,0,0.035989


In [673]:
(raw_indexes_list, updated_by_line_index) = get_raw_indexes_list(
  df_cann_lines_index=split_data['by_line_index']
)

In [674]:
raw_indexes_list

[[''],
 ['number',
  'italic refer',
  'text containing either',
  'de',
  'nition',
  'detailed discussion'],
 ['number', 'type refer', 'entry', 'glossary'],
 ['', 'mechanism', 'accident', 'luck analyticity'],
 ['ayer'],
 ['carnap'],
 ['boghossian'],
 ['bonjour', 'epistemic'],
 ['lewis', 'c', '', '', 'metaphysical', 'obfuscating conception'],
 ['quine',
  'reductive conception',
  '',
  'priori knowledge',
  'way knowing',
  'priori anti',
  'realism',
  'anti-realism'],
 ['prioricity', 'priori', 'knowledge priori', 'knowledge'],
 ['ayer'],
 ['bealer'],
 ['boghossian'],
 ['bonjour', 'criticism', 'empirical account', 'evaluativism', 'evaluativism'],
 ['kant'],
 ['kripke'],
 ['lewis', 'c', '', ''],
 ['maddy'],
 ['peacocke'],
 ['quine', 'set', 'theory', 'set-theory'],
 ['shapiro',
  'way',
  'knowing',
  'priori',
  'conceptual truth',
  'priorism',
  'knowledge',
  'priori arithmetic',
  'metaphysics',
  'mathematical',
  'object',
  'knowledge',
  'arithmetical',
  'independence',
  'r

In [675]:
save_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path,
  raw_indexes_list=raw_indexes_list
)

In [676]:
# clean_indexes


In [677]:
clean_indexes = load_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)

In [678]:
with_is_in_index = add_is_in_index(
  candidates_df=with_tfidf,
  indexes_list=clean_indexes
)

In [679]:
aggregated_df=aggregate_by_candidate(
  candidates_df=with_is_in_index  
)

In [680]:
print(len(aggregated_df.candidate_keyword.unique()))
aggregated_df.shape[0]

38436


38436

In [681]:
#CHANGEEEEE
save_aggregated_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    agg_df=aggregated_df,
    file_name="shapiro_97.pdf"
)

In [682]:
aggregated_df[aggregated_df.is_in_index==1].head(100)

,candidate_keyword,length,is_named_entity,is_named_author,is_in_toc,freq,is_in_index,tfidf,importance,position_in_context,POS
20,ability,7,0,0,0,0.000221,1,0.096447,0.420463,0.564397,NOUN
91,abstract object,15,0,0,0,0.000106,1,0.000000,0.485034,0.710565,CHUNK
103,acausal,7,0,0,0,0.000010,1,0.043647,0.706205,0.523810,PROPN
240,accident,8,0,0,0,0.000038,1,0.059134,0.558980,0.196994,NOUN
456,accuracy concept,16,0,0,0,0.000010,1,0.000000,0.564407,0.724138,CHUNK
464,accurate,8,0,0,0,0.000673,1,0.212637,0.549314,0.278555,ADJ
488,accurate representation,23,0,0,0,0.000067,1,0.000000,0.468959,0.615862,CHUNK
605,acquisition,11,0,0,0,0.000250,1,0.130752,0.500720,0.595700,NOUN
785,adjectival,10,0,0,0,0.000038,1,0.074900,0.584431,0.652155,NOUN
962,alan,4,1,0,0,0.000019,1,0.076776,0.416198,0.710084,PROPN
